In [188]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

import json, yaml

In [189]:
with open("openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)

In [190]:
def get_api_operation_by_id(json_spec, operation_id):
    # Convert the JSON string to a Python dictionary
    # spec_dict = json.loads(json_spec)

    # Traverse the dictionary to find the operation with the given ID
    paths = json_spec.dict_.get("paths", {})
    for path, methods in paths.items():
        for method, operation in methods.items():
            if operation.get("operationId") == operation_id:
                return operation

    # If the operation ID is not found, return None
    return None


In [191]:
# json_spec.dict_
api_operation=get_api_operation_by_id(json_spec, "createCompletion")

# api_operation
# # api_operation["responses"]["200"]["content"]["application/json"]

In [192]:
api_operation

{'operationId': 'createCompletion',
 'tags': ['OpenAI'],
 'summary': 'Creates a completion for the provided prompt and parameters.',
 'requestBody': {'required': True,
  'content': {'application/json': {'schema': {'$ref': '#/components/schemas/CreateCompletionRequest'}}}},
 'responses': {'200': {'description': 'OK',
   'content': {'application/json': {'schema': {'$ref': '#/components/schemas/CreateCompletionResponse'}}}}},
 'x-oaiMeta': {'name': 'Create completion',
  'returns': 'Returns a [completion](/docs/api-reference/completions/object) object, or a sequence of completion objects if the request is streamed.\n',
  'legacy': True,
  'examples': [{'title': 'No streaming',
    'request': {'curl': 'curl https://api.openai.com/v1/completions \\\n  -H "Content-Type: application/json" \\\n  -H "Authorization: Bearer $OPENAI_API_KEY" \\\n  -d \'{\n    "model": "VAR_model_id",\n    "prompt": "Say this is a test",\n    "max_tokens": 7,\n    "temperature": 0\n  }\'\n',
     'python': 'import 

In [193]:
api_operation["responses"]["200"]["content"]["application/json"]["schema"]

{'$ref': '#/components/schemas/CreateCompletionResponse'}

In [194]:
import re

def resolve_references(input_dict, json_spec):
    if not isinstance(input_dict, dict):
        return input_dict

    if "$ref" in input_dict and isinstance(input_dict["$ref"], str):
        match = re.match(r'#/components/schemas/(\w+)', input_dict["$ref"])
        if match:
            schema_name = match.group(1)
            if schema_name in json_spec.get("components", {}).get("schemas", {}):
                return json_spec["components"]["schemas"][schema_name]

    result = {}
    for key, value in input_dict.items():
        result[key] = resolve_references(value, json_spec)
    return result

def resolve_references_in_json_spec(json_spec):
    for key, value in json_spec.items():
        if isinstance(value, dict):
            json_spec[key] = resolve_references(value, json_spec)
    return json_spec

In [195]:

# Resolve references within the json_spec dictionary
json_spec = resolve_references_in_json_spec(json_spec.dict_)

input_dict = api_operation["requestBody"]["content"]["application/json"]["schema"]

result = resolve_references(input_dict, json_spec)
print(result)


{'type': 'object', 'properties': {'model': {'description': 'ID of the model to use. You can use the [List models](/docs/api-reference/models/list) API to see all of your available models, or see our [Model overview](/docs/models/overview) for descriptions of them.\n', 'anyOf': [{'type': 'string'}, {'type': 'string', 'enum': ['babbage-002', 'davinci-002', 'text-davinci-003', 'text-davinci-002', 'text-davinci-001', 'code-davinci-002', 'text-curie-001', 'text-babbage-001', 'text-ada-001']}], 'x-oaiTypeLabel': 'string'}, 'prompt': {'description': 'The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays.\n\nNote that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document.\n', 'default': '<|endoftext|>', 'nullable': True, 'oneOf': [{'type': 'string', 'default': '', 'example': 'This is a test.'}, {'type': '